# Trabajo Practico 1: Bayes Ingenuo

### Estudiantes:
1. Sophia Contreras
2. Yoksan Varela
3. Mauro Viquez

In [6]:
# Librerias usadas en el codigo
import torch
import torchvision
from PIL import Image
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torchvision.transforms as transforms

## Funciones Generales

In [7]:
def load_cifar10_dataset(is_train = True):
    """Funcion para cargar CIFAR10 dataset

    Args:
        is_train (bool, optional): Especifica si el modelo se esta entrenando. Defaults to True.

    Returns:
        Tensor: Retorna los tensores con sus respectivos labels
    """
    # Define a transformation to convert images to grayscale
    transforms_1 = transforms.Compose([
      transforms.ToTensor(),
      transforms.Grayscale(num_output_channels=1)  # Convert to grayscale
    ])
    cifar_trainset = datasets.CIFAR10(root='./data', train = is_train, download = True, transform = transforms_1)
  

    # Initialize an empty list to store batches
    all_data = []
    train_loader = torch.utils.data.DataLoader(cifar_trainset, batch_size = 64, shuffle=True)
    # Iterate over the train_loader to fetch all batches
    for batch in train_loader:
        images, _ = batch  # Extract images from the batch
        all_data.append(images)

    # Concatenate all batches into a single tensor along the batch dimension
    cifar_trainset_tensor = torch.round(torch.cat(all_data, dim=0) * 255)
    cifar_labels = torch.tensor(cifar_trainset.targets)
    print("cifar_trainset_tensor shape ", cifar_trainset_tensor.shape)
    print("cifar_labels ", cifar_labels.shape)
    return (cifar_trainset_tensor, cifar_labels)

In [8]:
# Cargando el dataset CIFAR10
load_cifar10_dataset()

100%|██████████| 170498071/170498071 [00:18<00:00, 9079266.34it/s] 


Extracting ./data\cifar-10-python.tar.gz to ./data
cifar_trainset_tensor shape  torch.Size([50000, 1, 32, 32])
cifar_labels  torch.Size([50000])


(tensor([[[[129., 128., 126.,  ..., 138., 137., 139.],
           [141., 139., 138.,  ..., 152., 149., 150.],
           [144., 139., 138.,  ..., 154., 148., 148.],
           ...,
           [231., 231., 236.,  ...,  62.,  36.,  58.],
           [236., 240., 240.,  ...,  96.,  53.,  55.],
           [236., 230., 225.,  ..., 115.,  78.,  57.]]],
 
 
         [[[170., 165., 178.,  ..., 217., 218., 204.],
           [169., 162., 173.,  ..., 198., 202., 168.],
           [164., 158., 165.,  ..., 164., 143., 168.],
           ...,
           [119., 119., 119.,  ..., 143., 144., 145.],
           [119., 120., 120.,  ..., 143., 144., 145.],
           [117., 117., 115.,  ..., 140., 140., 141.]]],
 
 
         [[[155., 120., 135.,  ..., 148., 154., 169.],
           [149., 113., 154.,  ..., 140., 147., 147.],
           [147., 126., 156.,  ..., 155., 159., 166.],
           ...,
           [191., 182., 174.,  ..., 167., 164., 158.],
           [182., 165., 163.,  ..., 158., 157., 159.],
     

In [ ]:
print(load_cifar10_dataset.shape)